In [1]:
#import all relvant libraries

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk import stem
stemmer = stem.PorterStemmer()
from nltk import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
import string
punct = list(string.punctuation)
from collections import Counter
import requests
import pandas as pd
!pip install PRAW
import numpy as np
import praw
import datetime
import hashlib

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\logan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\logan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\logan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#set request credentials to a variable which can be called upon 

reddit = praw.Reddit(user_agent='VAD',
                     client_id='3MelI32fEig1elS4ztSdkA', client_secret="mqWydYBCGDizuaBVIqeGfrJ8X7pr-A",
                     username='xxxxxxxxx', password='xxxxxxxxxx')

In [3]:
#checking all good. Test code to see if all the submission requests ect work

for submission in reddit.subreddit("test").hot(limit=10):
    print(submission.author)

kmiller0112
PuckElectra
all_about_everyone
PuzzleheadedYam5996
TextDetectorBot
streets-a-head
feelinn
Icarusmacsj1149
boringname101
jayfelee139


In [26]:
def submission(submission_id): ## submission_id can be URL or submission ID
    try:
        submission = reddit.submission(url = submission_id)
    except:
        submission = reddit.submission(submission_id)
        
    #collecting and presenting
    title = submission.title
    submission.comments.replace_more() ## loads new page if cooments are multipage
    text = [i.body for i in submission.comments]
    score = [i.score for i in submission.comments]
    user = [i.author for i in submission.comments]
    date = [datetime.datetime.fromtimestamp(i.created) for i in submission.comments]
    df = pd.DataFrame()
    df['text'] = text
    df['datetime'] = date
    df['score'] = score 
    df['subreddit'] = submission.subreddit
    df['redditor'] = user
    df['type'] = 'comment'
    df['title'] = title
    
    #prepping the data
    df['redditor_hashed'] = df['redditor'].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest()) ##anonymises the username 
    df['text_tokenised']= df['text'].apply(word_tokenize) #tokenises the comment
    df['text_lowered']= df['text_tokenised'].apply(lambda tok_list: [i.lower() for i in tok_list]) #puts it into lower case
    df['text_without_stopwords'] = [[word for word in row if word not in stops and word not in punct] for row in df['text_lowered']] #removes stopwords
    df['text_clean'] = [[word.encode('ascii', 'ignore').decode() for word in row if word not in stops and word not in punct] for row in df['text_lowered']] #converts everything to ASCII and ignores all characters that are unconvertable 
    df['text_lemmatised']= df['text_lowered'].apply(lambda lem_list: [lemmatizer.lemmatize(i) for i in lem_list]) #lemmatizes remaining data
    df = df.drop('redditor', axis=1)
    
    df = df.sort_values('score', ascending = False).reset_index(drop = True)
    return df

In [48]:
#testing the request and preparation of the data using the submission() function 
df = submission('https://www.reddit.com/r/Damnthatsinteresting/comments/jmln03/in_south_korea_the_solar_panels_in_the_middle_of/')

In [49]:
df

,text,datetime,score,subreddit,type,title,redditor_hashed,text_tokenised,text_lowered,text_without_stopwords,text_clean,text_lemmatised
0,"This idea is great. Unfortunately, I stay in F...",2020-11-02 12:01:56,5315,Damnthatsinteresting,comment,"In South Korea, the solar panels in the middle...",dc937b59892604f5a86ac96936cd7ff09e25f18ae6b758...,"[This, idea, is, great, ., Unfortunately, ,, I...","[this, idea, is, great, ., unfortunately, ,, i...","[idea, great, unfortunately, stay, florida, fe...","[idea, great, unfortunately, stay, florida, fe...","[this, idea, is, great, ., unfortunately, ,, i..."
1,I'm from SK but I've never seen this in my lif...,2020-11-02 13:02:00,1802,Damnthatsinteresting,comment,"In South Korea, the solar panels in the middle...",9f857e1a8855405dee584f3b0a2fac14c909642406de01...,"[I, 'm, from, SK, but, I, 've, never, seen, th...","[i, 'm, from, sk, but, i, 've, never, seen, th...","['m, sk, 've, never, seen, life, googling, app...","['m, sk, 've, never, seen, life, googling, app...","[i, 'm, from, sk, but, i, 've, never, seen, th..."
2,Okey.. nice. But how do you get on/off the bik...,2020-11-02 12:21:54,1387,Damnthatsinteresting,comment,"In South Korea, the solar panels in the middle...",a21e3373140cb2d76eb06608a684a14ee1ff8cbf522943...,"[Okey, .., nice, ., But, how, do, you, get, on...","[okey, .., nice, ., but, how, do, you, get, on...","[okey, .., nice, get, on/off, bike, path]","[okey, .., nice, get, on/off, bike, path]","[okey, .., nice, ., but, how, do, you, get, on..."
3,Americans: This is an amazing idea. \nDutch Gu...,2020-11-02 12:26:04,486,Damnthatsinteresting,comment,"In South Korea, the solar panels in the middle...",859600e9cf22435d4a71fa55fa336be183f38147cb013c...,"[Americans, :, This, is, an, amazing, idea, .,...","[americans, :, this, is, an, amazing, idea, .,...","[americans, amazing, idea, dutch, guy, exhaust...","[americans, amazing, idea, dutch, guy, exhaust...","[american, :, this, is, an, amazing, idea, ., ..."
4,We can't have that here. America is special in...,2020-11-02 11:37:46,366,Damnthatsinteresting,comment,"In South Korea, the solar panels in the middle...",490294b97dd9c899da4a4e5ba8331583b943d2e3d45e3a...,"[We, ca, n't, have, that, here, ., America, is...","[we, ca, n't, have, that, here, ., america, is...","[ca, n't, america, special, every, special, ci...","[ca, n't, america, special, every, special, ci...","[we, ca, n't, have, that, here, ., america, is..."
...,...,...,...,...,...,...,...,...,...,...,...,...
902,Miles ahead of the rest of the world,2020-11-02 12:30:24,-3,Damnthatsinteresting,comment,"In South Korea, the solar panels in the middle...",5a4c719dda49fb80c8395c1ed7182677c12b2955869cbe...,"[Miles, ahead, of, the, rest, of, the, world]","[miles, ahead, of, the, rest, of, the, world]","[miles, ahead, rest, world]","[miles, ahead, rest, world]","[mile, ahead, of, the, rest, of, the, world]"
903,"Comment section is full of Americans, again. R...",2020-11-02 13:35:14,-4,Damnthatsinteresting,comment,"In South Korea, the solar panels in the middle...",c201f7273bd55b72de573f8295c503a86259ad462427b2...,"[Comment, section, is, full, of, Americans, ,,...","[comment, section, is, full, of, americans, ,,...","[comment, section, full, americans, reddit, ne...","[comment, section, full, americans, reddit, ne...","[comment, section, is, full, of, american, ,, ..."
904,Its like totally logical. Will never happen i...,2020-11-02 12:13:22,-5,Damnthatsinteresting,comment,"In South Korea, the solar panels in the middle...",dc937b59892604f5a86ac96936cd7ff09e25f18ae6b758...,"[Its, like, totally, logical, ., Will, never, ...","[its, like, totally, logical, ., will, never, ...","[like, totally, logical, never, happen, merica]","[like, totally, logical, never, happen, merica]","[it, like, totally, logical, ., will, never, h..."
905,[removed],2020-11-02 13:27:23,-6,Damnthatsinteresting,comment,"In South Korea, the solar panels in the middle...",dc937b59892604f5a86ac96936cd7ff09e25f18

In [39]:
#add the URLs of the posts I want to extract comments from, to the list of the theme it relates to, so that when I itterate through, the  comments are seperated by post and aslo the request isn't as big  

solar_innovations = ['https://www.reddit.com/r/Damnthatsinteresting/comments/jmln03/in_south_korea_the_solar_panels_in_the_middle_of/' ,
                     'https://www.reddit.com/r/nextfuckinglevel/comments/ipei37/transparent_solar_panels_will_turn_windows_into/',
                     'https://www.reddit.com/r/technology/comments/fwlgim/oil_companies_are_collapsing_but_wind_and_solar/',
                     'https://www.reddit.com/r/science/comments/8h6uot/the_sea_slug_elysia_chlorotica_is_able_to_become/',
                     'https://www.reddit.com/r/science/comments/hgccyn/scientists_identify_a_novel_method_to_create/',
                     'https://www.reddit.com/r/Futurology/comments/7e5ir1/solar_power_is_now_the_cheapest_form_of_energy_on/',
                     'https://www.reddit.com/r/UpliftingNews/comments/10dc0iu/the_price_of_solar_panels_is_set_to_plunge/',
                     'https://www.reddit.com/r/energy/comments/114pzga/china_now_has_enough_wind_and_solar_to_power/']
                                 
Wind_power = ['https://www.reddit.com/r/worldnews/comments/ceejtt/scotland_is_now_generating_so_much_wind_energy_it/',
            'https://www.reddit.com/r/worldnews/comments/hze0sx/climate_crisis_offshore_wind_power_so_cheap_it/',
            'https://www.reddit.com/r/technology/comments/7ab7l1/there_was_so_much_wind_power_in_germany_this/',
            'https://www.reddit.com/r/worldnews/comments/ld5ebz/south_korea_unveils_43_billion_plan_for_worlds/',
            'https://www.reddit.com/r/news/comments/mfu8de/the_biden_administration_makes_a_swath_of_ocean/',
            'https://www.reddit.com/r/europe/comments/usxnn0/denmark_netherlands_germany_and_belgium_sign_135/',
            'https://www.reddit.com/r/Futurology/comments/xoj7sb/offshore_wind_125_times_better_for_taxpayers/']
    
Nuclear_Fission = ['https://www.reddit.com/r/unpopularopinion/comments/jj7y3h/if_you_actually_care_about_the_environment/',
                'https://www.reddit.com/r/science/comments/20b7v9/science_ama_series_were_professors_in_the/']
    
Nuclear_Fusion = ['https://www.reddit.com/r/technology/comments/zjfgxb/us_scientists_achieve_holy_grail_net_gain_nuclear/',
                'https://www.reddit.com/r/worldnews/comments/hzgwg4/fourteen_years_after_receiving_the_official/',
                'https://www.reddit.com/r/videos/comments/zl7vlx/nuclear_scientist_marv_adams_explains_what/',
                'https://www.reddit.com/r/technology/comments/wmwreq/nuclear_fusion_breakthrough_confirmed_california/',
                'https://www.reddit.com/r/Futurology/comments/wmj0kr/nuclear_fusion_ignition_confirmed_in_an/',
                'https://www.reddit.com/r/technology/comments/5grbo2/tests_confirm_that_germanys_massive_nuclear/']
                
Hydro_power = ['https://www.reddit.com/r/Damnthatsinteresting/comments/wqub34/the_belgian_startup_turbulent_hydro_has_invented/']
               


In [40]:
#Create a new DataFrame to which I can append the data structues as they go through the iteration

df_combined = pd.DataFrame()

In [53]:
#For loop to iterate through the links in the list and the print function is added so that I know it's working 
#The same code is used in the next few cells for each list

for url in solar_innovations:
    df = submission(url)
    df_combined_1 = pd.concat([df_combined,df])
    print('i have been added')


i have been added
i have been added
i have been added
i have been added
i have been added
i have been added
i have been added
i have been added


In [42]:
for url in Wind_power:
    df = submission(url)
    df_combined = pd.concat([df_combined,df])
    print('i have been added')


i have been added
i have been added
i have been added
i have been added
i have been added
i have been added
i have been added


In [43]:
for url in Nuclear_Fission:
    df = submission(url)
    df_combined = pd.concat([df_combined,df])
    print('i have been added')


i have been added
i have been added


In [44]:
for url in Nuclear_Fusion:
    df = submission(url)
    df_combined = pd.concat([df_combined,df])
    print('i have been added')


i have been added
i have been added
i have been added
i have been added
i have been added
i have been added


In [45]:
for url in Hydro_power:
    df = submission(url)
    df_combined = pd.concat([df_combined,df])
    print('i have been added')


i have been added


In [46]:
#print the DataFrame to see if everyting is good

df_combined

,text,datetime,score,subreddit,type,title,redditor_hashed,text_tokenised,text_lowered,text_without_stopwords,text_clean,text_lemmatised
0,"This idea is great. Unfortunately, I stay in F...",2020-11-02 12:01:56,5318,Damnthatsinteresting,comment,"In South Korea, the solar panels in the middle...",dc937b59892604f5a86ac96936cd7ff09e25f18ae6b758...,"[This, idea, is, great, ., Unfortunately, ,, I...","[this, idea, is, great, ., unfortunately, ,, i...","[idea, great, unfortunately, stay, florida, fe...","[idea, great, unfortunately, stay, florida, fe...","[this, idea, is, great, ., unfortunately, ,, i..."
1,I'm from SK but I've never seen this in my lif...,2020-11-02 13:02:00,1798,Damnthatsinteresting,comment,"In South Korea, the solar panels in the middle...",9f857e1a8855405dee584f3b0a2fac14c909642406de01...,"[I, 'm, from, SK, but, I, 've, never, seen, th...","[i, 'm, from, sk, but, i, 've, never, seen, th...","['m, sk, 've, never, seen, life, googling, app...","['m, sk, 've, never, seen, life, googling, app...","[i, 'm, from, sk, but, i, 've, never, seen, th..."
2,Okey.. nice. But how do you get on/off the bik...,2020-11-02 12:21:54,1397,Damnthatsinteresting,comment,"In South Korea, the solar panels in the middle...",a21e3373140cb2d76eb06608a684a14ee1ff8cbf522943...,"[Okey, .., nice, ., But, how, do, you, get, on...","[okey, .., nice, ., but, how, do, you, get, on...","[okey, .., nice, get, on/off, bike, path]","[okey, .., nice, get, on/off, bike, path]","[okey, .., nice, ., but, how, do, you, get, on..."
3,Americans: This is an amazing idea. \nDutch Gu...,2020-11-02 12:26:04,483,Damnthatsinteresting,comment,"In South Korea, the solar panels in the middle...",859600e9cf22435d4a71fa55fa336be183f38147cb013c...,"[Americans, :, This, is, an, amazing, idea, .,...","[americans, :, this, is, an, amazing, idea, .,...","[americans, amazing, idea, dutch, guy, exhaust...","[americans, amazing, idea, dutch, guy, exhaust...","[american, :, this, is, an, amazing, idea, ., ..."
4,We can't have that here. America is special in...,2020-11-02 11:37:46,371,Damnthatsinteresting,comment,"In South Korea, the solar panels in the middle...",490294b97dd9c899da4a4e5ba8331583b943d2e3d45e3a...,"[We, ca, n't, have, that, here, ., America, is...","[we, ca, n't, have, that, here, ., america, is...","[ca, n't, america, special, every, special, ci...","[ca, n't, america, special, every, special, ci...","[we, ca, n't, have, that, here, ., america, is..."
...,...,...,...,...,...,...,...,...,...,...,...,...
727,Oh bro that’s so fucking bad. Just build nucle...,2022-08-18 02:51:48,-2,Damnthatsinteresting,comment,The Belgian start-up Turbulent Hydro has inven...,dc937b59892604f5a86ac96936cd7ff09e25f18ae6b758...,"[Oh, bro, that, ’, s, so, fucking, bad, ., Jus...","[oh, bro, that, ’, s, so, fucking, bad, ., jus...","[oh, bro, ’, fucking, bad, build, nuclear, lmao]","[oh, bro, , fucking, bad, build, nuclear, lmao]","[oh, bro, that, ’, s, so, fucking, bad, ., jus..."
728,What they don't mention is that there's usuall...,2022-08-17 19:52:25,-3,Damnthatsinteresting,comment,The Belgian start-up Turbulent Hydro has inven...,063f1c9669eb28a641b7336e32fee23838f2edd8bba381...,"[What, they, do, n't, mention, is, that, there...","[what, they, do, n't, mention, is, that, there...","[n't, mention, 's, usually, co2, plume, around...","[n't, mention, 's, usually, co2, plume, around...","[what, they, do, n't, mention, is, that, there..."
729,"""It's such a shame the inventors of this devic...",2022-08-17 19:28:36,-4,Damnthatsinteresting,comment,The Belgian start-up Turbulent Hydro has inven...,764530f6e0ce5208063f47c4ed28126e30f6bacfb80d1f...,"[``, It, 's, such, a, shame, the, inventors, o...","[``, it, 's, such, a, shame, the, inventors, o...","[``, 's, shame, inventors, device, killed, two...","[``, 's, shame, inventors, device, killed, two...","[``, it, 's, such, a, shame, the, inventor, of..."
730,“Without rivers”…..(then insert picture of riv...,2022-08-17 18:18:24,-10,Damnthatsint

In [60]:
#if wanted in Excel run this cell

df_combined.to_excel('Summative_1_DataFrame_21008600514.xlsx')

In [59]:
#if wanted in CSV run this cell 

df_combined.to_csv('Summative_1_DataFrame_21008600514.csv')

To access the Data, 
Enter your reddit authentication details in the second cell, then run the whole programme and either of the cells above to get the code in the desired format.


In [58]:
#500 word description below 




For this project the long term plan is to gain an insight into the attitudes surrounding different renewable alternatives to non-renewable energy sources. For this I will be using reddit to find posts mentioning different alternative energy sources namely Solar, Wind, Nuclear Fusion, Nuclear fission, Hydropower and analyse the comments around each one to determine what the sentiment around the different sources are currently. 
To do collect the data, I first searched through reddit to find the posts that have the most upvotes and most comments. Then I selected the posts that actually spoke about the application or innovations in the alternative energy source I was looking at which I added to a list. Unfortunately not all the different categories I would like to analyse have the same amount of posts or comments which could affect the validity of the findings
I then wrote the code to be able to extract the data from reddit. I started by importing all the libraries required to obtain the data and process it for NLP. Key Libraries used in this project are PRAW, Pandas, NLTK. After uploading my authentication information, I ran a quick test code to see if everything was working
I then started on working on  a function that was previously given as an example to only extract information from a post on reddit. I used that as base code to build off. Based off the basic DataFrame the base code provides, I firstly added a line of code which instantly anonymised the redditors usernames using sha256 encoding. I then tokenised the data and added it to a separate column so that I could track the work the code was doing to make sure it was doing the right things. I then set all the items to lower case. I did this first because the list of stop words are all in lower case. If I had not set all the words to lower case, there would still be stop words in the data set. I then proceeded to data points of anything value that isn’t a word by encoding all the values into ASCII and ignoring whatever can’t be encoded and decoding it again to remove all unwanted characters. Then I simply lemmatised the remaining data. All these different steps were all added to separate columns in the DataFrame so that the progression the cleaning process can be seen. The Data in the ‘text_lemmatised’ column is all ready for NLP analysis. The function set up so that any link put into it is ready for NLP analysis.
I then created multiple for loops which iterated through the list of links provided to process the data and append it to dataframe called df_combined which contains all the comments extracted through the API. For further NLP analysis I intend on expanding the number of posts and comments on each topic.
